In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns 
from scipy.stats import chi2_contingency

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#from pathlib import Path
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
#Import Dataset to Pandas Dataframe
dataset = pd.read_csv(os.path.join(dirname, filename))

In [ ]:
dataset.head()

In [ ]:
dataset.shape 

In [ ]:
headers = ["ID", "Gender", "Age", "Hypertension", "Heart_Disease", "Ever_Married", "Work_Type", 
           "Residence_Type", "Avg_Glucose_Level", "BMI", "Smoking_Status", "Stroke"]

dataset.columns = headers

dataset.head()

In [ ]:
# datasetinin özellikleri
dataset.info()

In [ ]:
#datasetinde yer alan nümerik verilerin istatistiksel bilgileri. 
dataset.describe()

In [ ]:
#Sayısal olmayan tüm sütunların istatiksel bilgileri.
dataset.describe(include=['O'])

# EKSİK VERİ KONTROLÜ

In [ ]:
#null veri kontrolü
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
#null verilerin setten silinmesi
dataset.dropna(axis=0, inplace=True)

dataset.reset_index(drop=True, inplace=True)
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
# datasetinin özellikleri
dataset.info()

# AYKIRI VERİ KONTROLÜ

In [ ]:
#Unique verilerin kontrolü
dataset.nunique()

* #Gender - 3 veri tipi olduğu için kontrol edilmelidir. 
* #Hypertension, Heart Disease, Ever_married, Residence_Type, Stroke - Yes ve No ya da  1-0 şeklinde olabilir.
* #Smoking Status (4) ve Work Type (5) Kategorik veri olabilir. 

In [ ]:
dataset.Gender.unique()

In [ ]:
dataset.Gender.value_counts()

Gender - Other dan sadece 1 adet veri girişi olduğundan dolayı aykırı veridir. Bu yüzden bu veriyi silebiliriz.

In [ ]:
#Gender sütunundan Other satırının silinmesi
dataset=dataset[dataset['Gender']!='Other'] 
dataset.Gender.value_counts()

# KESİKLİ VERİYE DÖNÜŞTÜRME

In [ ]:
#Kesikli Veriye Dönüştürme
dataset['BMI_Category'] = pd.cut(dataset['BMI'], bins = [0, 18.5, 25,30,35,40,10000],
                            labels = ['Underweight', 'Normal Weight', 'Overweight', 
                                      'Obesity Class I','Obesity Class II','Obesity Class III'])

dataset['Age_Category'] = pd.cut(dataset['Age'], bins = [0,13,18, 45,60,100], 
                                 labels = ['Children', 'Teens', 'Adults','Mid Adults','Elderly'])

dataset['BloodGlucose_Category'] = pd.cut(dataset['Avg_Glucose_Level'], bins = [0,90,140,200,500],
                                          labels = ['Low', 'Normal', 'Prediabetic', 'Diabetic'])

BMI, Age ve Glucose Level sürekli türden verilerdir.Bu verileri sınıflandırarak kesikli hale getiriyoruz.

In [ ]:
dataset.head()

**BMI	Nutritional status**
* Below 18.5  Underweight
* 18.5–24.9  Normal weight
* 25.0–29.9  Pre-obesity
* 30.0–34.9  Obesity class I
* 35.0–39.9  Obesity class II
* Above 40  Obesity class III

Kaynak: https://www.euro.who.int/en/health-topics/disease-prevention/nutrition/a-healthy-lifestyle/body-mass-index-bmi)

**AVERAGE GLUCOSE LEVELS**
* A blood sugar level less than 140 mg/dL (7.8 mmol/L) is normal. 
* A reading of more than 200 mg/dL (11.1 mmol/L) after two hours indicates diabetes. 
* A reading between 140 and 199 mg/dL (7.8 mmol/L and 11.0 mmol/L) indicates prediabetes. 

Kaynak: https://www.mayoclinic.org/diseases-conditions/diabetes/diagnosis-treatment/drc-0371451#:~:text=A%20blood%20sample%20will%20be%20taken%20after%20an%20overnight%20fast,separate%20tests%2C%20you%20have%20diabetes.

* Less than 90 - low
* 90 - 140 - normal
* 140 - 199 - prediabetic
* 200 and more - diabetic

In [ ]:
dataset.Work_Type.unique()

In [ ]:
def work_type_encoder(i):
    if i=='Self-employed':
        return 2
    elif i in ['Private','Govt_job']:
        return 1
    else: 
        return 0

dataset["Work_Type"]=dataset["Work_Type"].apply(work_type_encoder)

In [ ]:
unknown_smokers = dataset[dataset['Smoking_Status']=='Unknown']

unknown_smokers.Age_Category.value_counts(normalize=True)

In [ ]:
married = dataset[dataset['Ever_Married']==1]

married.Age_Category.value_counts(normalize=True)

In [ ]:
stroke = dataset[dataset['Stroke']==1]

stroke.Age_Category.value_counts(normalize=True)

Çoğunluğu children kategorisinde olduğu için unknown 0 olarak güncelleniyor.

In [ ]:
#Sayısal olmayan veriler sayısala dönüştürülüyor.
#Gender
dataset.loc[(dataset.Gender == 'Male'),'Gender'] = 1
dataset.loc[(dataset.Gender == 'Female'),'Gender'] = 0

# Ever_Married
dataset.loc[(dataset.Ever_Married == 'Yes'),'Ever_Married'] = 1
dataset.loc[(dataset.Ever_Married == 'No'),'Ever_Married'] = 0

# Residence_Type
dataset.loc[(dataset.Residence_Type == 'Urban'),'Residence_Type'] = 1
dataset.loc[(dataset.Residence_Type == 'Rural'),'Residence_Type'] = 0

# Smoking_Status

dataset.loc[(dataset.Smoking_Status == 'formerly smoked'),'Smoking_Status'] = 1
dataset.loc[(dataset.Smoking_Status == 'never smoked'),'Smoking_Status'] = 0
dataset.loc[(dataset.Smoking_Status == 'smokes'),'Smoking_Status'] = 1
dataset.loc[(dataset.Smoking_Status == 'Unknown'),'Smoking_Status'] = 0


dataset[["Gender", "Ever_Married", "Residence_Type","Smoking_Status"]] = dataset[["Gender", "Ever_Married", "Residence_Type","Smoking_Status"]].apply(pd.to_numeric)

# KORELASYON ANALİZİ

In [ ]:
#Dataset korelasyon analizi (sayısal nitelikler)
dataset.corr()

In [ ]:
#Korelasyon tablosunun heatmap grafiği
corr = dataset.corr()
mask = np.triu(np.ones_like(corr, dtype=np.bool))
f, ax = plt.subplots(figsize=(10, 10))
cmap = sns.diverging_palette(250, 25, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=None, center=0,square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .9})

Evlilik ve stroke arasındaki korelasyonun yüksek olmasının nedeni evlilik ve yaş arasındaki korelasyonun yüksek olmasından dolayı olabilir.

# **Kİ KARE**

In [ ]:
contigency = pd.crosstab(dataset['Gender'],dataset['Stroke'])
contigency
import scipy.stats as stats
observed_val = contigency.values
print('Observed values: \n',observed_val,'\n')
val = stats.chi2_contingency(contigency)
expected_val = val[3]
print('Expected values: \n',expected_val,'\n')
alpha=0.05
rows = len(contigency.iloc[0:5,0])
cols = len(contigency.iloc[0 , 0:2])

ddof = (rows-1)*(cols-1)
print("Degree of freedom: ",ddof)

from scipy.stats import chi2
chi_s = sum([(o-e)**2./e for o,e in zip(observed_val,expected_val)])
chi_square_statistic = chi_s[0]+chi_s[1]
print("Chi square Statistic: ",chi_square_statistic)
critical_value = chi2.ppf(q=1-alpha,df=ddof)
print("Critical Value: ",critical_value,'\n')

if chi_square_statistic >= critical_value:
    print('Reject H0, There is a relationship between 2 categorical values. (chi_square_statistic >= critical_value)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (chi_square_statistic < critical_value)','\n')

p_value = 1-chi2.cdf(x=chi_square_statistic,df=ddof)
print("p value: ",p_value)

print("Alpha: ",alpha,'\n')

if p_value <= alpha:
    print('Reject H0, There is a relationship between 2 categorical values. (p_value <= alpha)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (p_value > alpha)','\n')

In [ ]:
contigency = pd.crosstab(dataset['Age_Category'],dataset['Stroke'])
contigency
import scipy.stats as stats
observed_val = contigency.values
print('Observed values: \n',observed_val,'\n')
val = stats.chi2_contingency(contigency)
expected_val = val[3]
print('Expected values: \n',expected_val,'\n')
alpha=0.05
rows = len(contigency.iloc[0:5,0])
cols = len(contigency.iloc[0 , 0:2])

ddof = (rows-1)*(cols-1)
print("Degree of freedom: ",ddof)

from scipy.stats import chi2
chi_s = sum([(o-e)**2./e for o,e in zip(observed_val,expected_val)])
chi_square_statistic = chi_s[0]+chi_s[1]
print("Chi square Statistic: ",chi_square_statistic)
critical_value = chi2.ppf(q=1-alpha,df=ddof)
print("Critical Value: ",critical_value,'\n')

if chi_square_statistic >= critical_value:
    print('Reject H0, There is a relationship between 2 categorical values. (chi_square_statistic >= critical_value)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (chi_square_statistic < critical_value)','\n')

p_value = 1-chi2.cdf(x=chi_square_statistic,df=ddof)
print("p value: ",p_value)

print("Alpha: ",alpha,'\n')

if p_value <= alpha:
    print('Reject H0, There is a relationship between 2 categorical values. (p_value <= alpha)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (p_value > alpha)','\n')

In [ ]:
contigency = pd.crosstab(dataset['BloodGlucose_Category'],dataset['Stroke'])
contigency
import scipy.stats as stats
observed_val = contigency.values
print('Observed values: \n',observed_val,'\n')
val = stats.chi2_contingency(contigency)
expected_val = val[3]
print('Expected values: \n',expected_val,'\n')
alpha=0.05
rows = len(contigency.iloc[0:5,0])
cols = len(contigency.iloc[0 , 0:2])

ddof = (rows-1)*(cols-1)
print("Degree of freedom: ",ddof)

from scipy.stats import chi2
chi_s = sum([(o-e)**2./e for o,e in zip(observed_val,expected_val)])
chi_square_statistic = chi_s[0]+chi_s[1]
print("Chi square Statistic: ",chi_square_statistic)
critical_value = chi2.ppf(q=1-alpha,df=ddof)
print("Critical Value: ",critical_value,'\n')

if chi_square_statistic >= critical_value:
    print('Reject H0, There is a relationship between 2 categorical values. (chi_square_statistic >= critical_value)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (chi_square_statistic < critical_value)','\n')

p_value = 1-chi2.cdf(x=chi_square_statistic,df=ddof)
print("p value: ",p_value)

print("Alpha: ",alpha,'\n')

if p_value <= alpha:
    print('Reject H0, There is a relationship between 2 categorical values. (p_value <= alpha)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (p_value > alpha)','\n')

In [ ]:
contigency = pd.crosstab(dataset['BMI_Category'],dataset['Stroke'])
contigency
import scipy.stats as stats
observed_val = contigency.values
print('Observed values: \n',observed_val,'\n')
val = stats.chi2_contingency(contigency)
expected_val = val[3]
print('Expected values: \n',expected_val,'\n')
alpha=0.05
rows = len(contigency.iloc[0:7,0])
cols = len(contigency.iloc[0 , 0:2])

ddof = (rows-1)*(cols-1)
print("Degree of freedom: ",ddof)

from scipy.stats import chi2
chi_s = sum([(o-e)**2./e for o,e in zip(observed_val,expected_val)])
chi_square_statistic = chi_s[0]+chi_s[1]
print("Chi square Statistic: ",chi_square_statistic)
critical_value = chi2.ppf(q=1-alpha,df=ddof)
print("Critical Value: ",critical_value,'\n')

if chi_square_statistic >= critical_value:
    print('Reject H0, There is a relationship between 2 categorical values. (chi_square_statistic >= critical_value)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (chi_square_statistic < critical_value)','\n')

p_value = 1-chi2.cdf(x=chi_square_statistic,df=ddof)
print("p value: ",p_value)

print("Alpha: ",alpha,'\n')

if p_value <= alpha:
    print('Reject H0, There is a relationship between 2 categorical values. (p_value <= alpha)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (p_value > alpha)','\n')

In [ ]:
contigency = pd.crosstab(dataset['Smoking_Status'],dataset['Stroke'])
contigency
import scipy.stats as stats
observed_val = contigency.values
print('Observed values: \n',observed_val,'\n')
val = stats.chi2_contingency(contigency)
expected_val = val[3]
print('Expected values: \n',expected_val,'\n')
alpha=0.05
rows = len(contigency.iloc[0:5,0])
cols = len(contigency.iloc[0 , 0:2])

ddof = (rows-1)*(cols-1)
print("Degree of freedom: ",ddof)

from scipy.stats import chi2
chi_s = sum([(o-e)**2./e for o,e in zip(observed_val,expected_val)])
chi_square_statistic = chi_s[0]+chi_s[1]
print("Chi square Statistic: ",chi_square_statistic)
critical_value = chi2.ppf(q=1-alpha,df=ddof)
print("Critical Value: ",critical_value,'\n')

if chi_square_statistic >= critical_value:
    print('Reject H0, There is a relationship between 2 categorical values. (chi_square_statistic >= critical_value)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (chi_square_statistic < critical_value)','\n')

p_value = 1-chi2.cdf(x=chi_square_statistic,df=ddof)
print("p value: ",p_value)

print("Alpha: ",alpha,'\n')

if p_value <= alpha:
    print('Reject H0, There is a relationship between 2 categorical values. (p_value <= alpha)','\n')
else:
    print('Retain H0, There is no relationship between 2 categorical values. (p_value > alpha)','\n')

# VERİ GÖRSELLEŞTİRME

In [ ]:
Stroke_plot = dataset['Stroke'].value_counts().reset_index()
Stroke_plot.columns = ['Stroke','Count']

px.pie(Stroke_plot,values='Count',names='Stroke',template='plotly',title='Stroke')

In [ ]:
fig,axes = plt.subplots(6,2,figsize = (30,30))
sns.set_style('whitegrid')
fig.suptitle("Count plot for various categorical features")

sns.countplot(ax=axes[0,0],data=dataset,x='Gender')
sns.countplot(ax=axes[0,1],data=dataset,x='Age')
sns.countplot(ax=axes[1,0],data=dataset,x='Age_Category')
sns.countplot(ax=axes[1,1],data=dataset,x='Hypertension')
sns.countplot(ax=axes[2,0],data=dataset,x='Heart_Disease')
sns.countplot(ax=axes[2,1],data=dataset,x='Ever_Married')
sns.countplot(ax=axes[3,0],data=dataset,x='Work_Type')
sns.countplot(ax=axes[3,1],data=dataset,x='Residence_Type')
sns.countplot(ax=axes[4,0],data=dataset,x='Smoking_Status')
sns.countplot(ax=axes[4,1],data=dataset,x='Stroke')
sns.countplot(ax=axes[5,0],data=dataset,x='BMI_Category')
sns.countplot(ax=axes[5,1],data=dataset,x='BloodGlucose_Category')

plt.show()


# NAIVE BAYES

In [ ]:
#Kategorik veriye dönüştürülen sütunlar datasetten siliniyor. 
dataset.drop(['Age_Category'], axis=1,inplace=True)
dataset.drop(['BMI_Category'],axis=1,inplace=True)
dataset.drop(['BloodGlucose_Category'], axis=1,inplace=True)
#identifer siliniyor.
dataset.drop(['ID'], axis=1,inplace=True)

In [ ]:
dataset.head()

In [ ]:
from sklearn.model_selection import train_test_split
y = dataset['Stroke']
x = dataset.drop(['Stroke'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=0, stratify=y)

In [ ]:
x.head()

In [ ]:
# Testing with GNB Algorithm
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(x_train, y_train)

gnb_pred = GNB.predict(x_test)
from sklearn import metrics
print(metrics.plot_confusion_matrix(GNB, x_test, y_test))
print(metrics.classification_report(y_test, gnb_pred, digits=3))

# BAGGING NAIVE BAYES

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report

seed = 50
num_trees =100
kfold = model_selection.KFold(n_splits=5, random_state=seed,shuffle=True)

NB = GaussianNB()

clf_array = [NB]

for clf in clf_array:
    
    bagging_clf = BaggingClassifier(base_estimator=clf,n_estimators=num_trees, random_state=seed).fit(x_train,y_train) 
    bagging_scores = model_selection.cross_val_score(bagging_clf, x_train, y_train, cv=10, n_jobs=1)#, error_score='raise')
    
    print ("Mean of: {1:.3f}, std: (+/-) {2:.3f} [Bagging {0}]\n".format(clf.__class__.__name__,bagging_scores.mean(), bagging_scores.std()))
    predicted = model_selection.cross_val_predict(bagging_clf,x_test,y_test,cv=kfold) 
    print ("Test classification_report \n", classification_report(y_test, predicted,digits=3))
    from sklearn import metrics
    NB.fit(x_train, y_train)
    print(metrics.plot_confusion_matrix(NB, x_test, y_test))
    